In [32]:
# 1. Imports and topic setup
from newsapi import NewsApiClient
from newsplease import NewsPlease
from sqlalchemy import create_engine, ForeignKey, Column, Integer, String, Text, select
from sqlalchemy.orm import declarative_base, relationship, Session
from articles_orm import Base, Article, ArticleFeatures
from more_itertools import unique_everseen

from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag

newsapi = NewsApiClient(api_key='fbfb692eb3844ce59e10eea6069d1161')

topic_list = ['crypto',
              'bitcoin',
              
             'covid',
             'vaccine',
              
             'startups',
              'technology',
              'e-commerce',
              
             'Nokia',
             'Apple',
             'Microsoft']

In [24]:
# 2. Pull 100 initial articles of all chosen topics
# NewsAPI doesn't allow free accounts to access results after number 100.
# NewsAPI also doesn't allow free accounts to access results from before >1month ago.
all_articles = {topic: newsapi.get_everything(q=topic, page_size=100, page=1, language='en') for topic in topic_list}
print(all_articles.keys())

dict_keys(['crypto', 'bitcoin', 'covid', 'vaccine', 'startups', 'technology', 'e-commerce', 'Nokia', 'Apple', 'Microsoft'])


In [25]:
# 3. Check totalResults num and length of downloaded content for integrity
print(*(all_articles[topic]['totalResults'] for topic in topic_list))
print(*(len(all_articles[topic]['articles']) for topic in topic_list))

11119 13186 213914 60707 5777 107836 11419 1020 39327 19633
100 100 100 100 100 100 100 100 100 100


In [26]:
# 4. Save urls of every topic to their own dict, but only unique articles (previously some duplicates appeared)
topic_urls = {}
for topic in topic_list:
    cta = all_articles[topic]['articles']
    current_topic_articles = []
    current_topic_articles_titles = []
    for article in cta: 
        if article['title'] not in current_topic_articles_titles: # save only unique
            current_topic_articles += [article]
            current_topic_articles_titles += [article['title']]
    
    current_topic_urls = [article['url'] for article in current_topic_articles]
    topic_urls[topic] = current_topic_urls
print(topic_urls.keys())
print(len(topic_urls['bitcoin']))

dict_keys(['crypto', 'bitcoin', 'covid', 'vaccine', 'startups', 'technology', 'e-commerce', 'Nokia', 'Apple', 'Microsoft'])
94


In [27]:
# 5. Establish session to database.
engine = create_engine(f"sqlite:///NewsAPI_articles.db")

In [28]:
# Create schema
Base.metadata.create_all(engine)

In [29]:
# Establish db session, get maintext of article, save objects
with Session(engine) as session:
    objects = []
    for topic in topic_list:
        for url in topic_urls[topic]:
            try:
                article = NewsPlease.from_url(url)
            except: # some download error happened, skip this article
                continue
            if (article.maintext is not None):
                mdl_article = Article(
                    parent_topic = topic,
                    source = article.source_domain,
                    title = article.title,
                    description = article.description,
                    maintext = article.maintext,
                    publication_date = article.date_publish,
                    url = url
                )
                print(mdl_article)
                objects += [mdl_article]
        session.bulk_save_objects(objects) # bulk save all topics then flush
        session.commit()
        objects=[]

Article(topic=crypto, title=Frst and Fabric Ventures announce fellowship program for crypto entrepreneurs, description=VC funds Frst and Fabric Ventures are teaming up to create Le Crypto Fellowship. With this program, the two firms want to find the next 10 crypto entrepreneurs in France. And they think they might foster the most promising crypto startups if they don’t have any preconceived idea and team yet. As Pierre Entremont from […] url=http://techcrunch.com/2021/06/03/frst-and-fabric-ventures-announce-fellowship-program-for-crypto-entrepreneurs/)


D:\anaconda3\envs\pracamgr\lib\site-packages\dateutil\parser\_parser.py:1213: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


Article(topic=crypto, title=Crypto exchange Binance faces US money laundering probe, description=Binance, the world's leading cryptocurrency exchange, is reportedly being probed by the DoJ and IRS amid allegations that it is being used to launder money.. url=https://www.engadget.com/crypto-exchange-binance-money-laundering-140037194.html)
Article(topic=crypto, title=Most new NVIDIA RTX gaming GPUs will be crypto-limited, description=NVIDIA is extending its limits on GPU capacity to more GeForce RTX 30-series cards in a bid to limit the shortages caused by cryptomining.. url=https://www.engadget.com/nvidia-rtx-gpu-crypto-limit-122007238.html)
Article(topic=crypto, title=In search of a new crypto deity, description=Hello friends, and welcome back to Week in Review! Last week, I wrote about tech taking on Disney. This week, I’m talking about the search for a new crypto messiah. If you’re reading this on the TechCrunch site, you can get this in your inbox from the newsletter page, and foll

Article(topic=crypto, title=Announcing the Disrupt 2021 Extra Crunch Stage Agenda, description=Disrupt 2020 was an absolute banger of a conference, but we’re not ones to rest on our laurels. We have an incredible Disrupt in store for you this September 21-23, with an amazing lineup of speakers across the hottest of topics, like crypto, health/biotech, SPACs, the creator economy and the usual spread of startup how-to […] url=http://techcrunch.com/2021/06/03/announcing-the-disrupt-2021-extra-crunch-stage-agenda/)
Article(topic=crypto, title=Art app SketchAR to allow artists to list their artworks on NFT marketplaces directly, description=An update to an existing art app that allows artists to access NFT marketplaces directly could have the potential to democratize access to the NFT world for artists not currently in crypto. SketchAR is an existing mobile app that allows artists to turn photos into illustrations using its AI-based computer vision. It is now launching […] url=http://techcr

Article(topic=crypto, title=El Salvador becomes the first country to approve Bitcoin as legal currency, description=El Salvador has voted to adopt Bitcoin as legal tender.. url=https://www.engadget.com/el-salvador-bitcoin-legal-tender-120440725.html)
Article(topic=crypto, title=Opera brings its gaming browser to mobile, description=Opera has begun beta testing its gaming browser on mobile with plans for a public launch in the coming weeks.. url=https://www.engadget.com/opera-gx-mobile-gaming-browser-114556643.html)
Article(topic=crypto, title=A Peter Thiel-backed blockchain platform just invested $10 billion in a new crypto exchange, description=Peter Thiel. AGON Limited Block.one announced on Tuesday that it would launch a crypto exchange called Bullish. It's landed over $10 billi... url=https://markets.businessinsider.com/currencies/news/peter-thiel-blockchain-platform-invested-billions-crypto-exchange-mike-novogratz-2021-5-1030415884)
Article(topic=crypto, title=Ray Dalio says crypt

...

Article(topic=Microsoft, title=Microsoft’s Surface Duo gets dual-screen gaming support, description=In December, Microsoft wrote a blog post highlighting “The year ahead for Surface Duo.” The news highlighted the dual-screen device’s upcoming availability outside the U.S. (including Canada, the U.K., France and Germany this year), as well as a smattering of features. It’s been pretty well acknowledged from most everyone who reviewed the device (us included), […] url=http://techcrunch.com/2021/05/24/microsofts-surface-duo-gets-dual-screen-gaming-support/)
Article(topic=Microsoft, title=Cruise can now give passengers rides in driverless cars in California, description=Cruise, the autonomous vehicle subsidiary of GM that also has backing from SoftBank Vision Fund, Microsoft and Honda, has secured a permit that will allow the company to shuttle passengers in its test vehicles without a human safety operator behind the wheel. The permit, issued by the California Public Utilities Commission 

Article(topic=Microsoft, title=Xbox Game Pass adds bottom screen touch controls on Surface Duo, description=Xbox Game Pass on Microsoft Surface Duo will let you play games with touch controls entirely on the bottom display so your thumbs won't block the action. url=https://mashable.com/article/microsoft-xbox-game-pass-surface-duo-touch-controls/)
Article(topic=Microsoft, title=Microsoft To Introduce New Version Of Windows On June 24, description=Microsoft (NASDAQ:MSFT) has announced that it will unveil the next generation of its Windows computer operating system on June 24. 

Windows, the dominant operating system for personal computers around the world, accounts for 14% of Microsoft’s total ... url=https://www.baystreet.ca/techinsider/2937/Microsoft-To-Introduce-New-Version-Of-Windows-On-June-24)
Article(topic=Microsoft, title=Bill Gates investigated by Microsoft before leaving board: report, description=The new report found that Gates resigned before the board's investigation into al

not a 200 response: 503


Article(topic=Microsoft, title=Microsoft announces retirement of Internet Explorer, description=Microsoft announces retirement of Internet Explorer url=https://www.rfi.fr/en/business-and-tech/20210521-microsoft-announces-retirement-of-internet-explorer)
Article(topic=Microsoft, title=91% off online class (UK deal), description=The Complete Big Data and Power BI Bundle is on sale for £28.28 as of May 13, saving you 91% on list price. url=https://mashable.com/uk/shopping/may-13-big-data-online-course-sale/)
Article(topic=Microsoft, title=Microsoft Xbox Game Pass gets support for Surface Duo, description=Microsoft has announced that the Xbox Game Pass app for Android now officially supports the Surface Duo and its unique dual-screen setup. The company has url=https://www.canindia.com/microsoft-xbox-game-pass-gets-support-for-surface-duo/)
Article(topic=Microsoft, title=Apple and Microsoft's Rivalry Had Cooled. Now It's Back and Getting Testier, description=After collaborating on various p

Article(topic=Microsoft, title=Microsoft Has Turned the Surface Duo Into a Handheld Xbox, description=Microsoft is updating its Xbox Cloud Gaming (xCloud) app for Android, and it includes dual-screen support for the Surface Duo. The Verge reports: The app update allows Surface Duo owners to use a virtual gamepad on one screen of their device and games on the other. It makes the Surface Duo look mor... url=https://games.slashdot.org/story/21/05/24/2015213/microsoft-has-turned-the-surface-duo-into-a-handheld-xbox)
Article(topic=Microsoft, title=Microsoft Says 'Next Generation of Windows' Coming Soon, description=Microsoft CEO Satya Nadella, at the company's developer conference today: Across all the opportunities I've highlighted today, Windows is implicit. It's never been more important. Windows 10 is used by more than 1.3 billion people to work, learn, connect and play. And it all starts with Windows as ... url=https://tech.slashdot.org/story/21/05/25/1833203/microsoft-says-next-genera

In [30]:
# Query all articles and check everything ids
with Session(engine) as session:
    articles = session.query(Article).all()
print(len(articles))
print(articles[-1].id)

962
962


In [42]:
import nltk
nltk.download('averaged_perceptron_tagger')

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif tag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:
        return nltk.corpus.wordnet.NOUN # fallback to NOUN for all else

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\szcze\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [45]:
# For every article from db, apply feature extraction methods (preprocessing -> bag of words -> pca/nmf -> save most important features as ArticleFeatures)
additional_remove_chars = "'\"`—“’”"
punctuation = string.punctuation.replace('-','')+additional_remove_chars
wordnet_lemmatizer = WordNetLemmatizer()
for article in articles:
    # preprocess
    lemmas = []
    article_text = article.maintext.lower()
    article_text = article_text.translate(str.maketrans('', '', punctuation)) # fastest strip punctuation
    article_tokens = word_tokenize(article_text)
    article_tokens = pos_tag(article_tokens) # tag proper pos based on context in sentence
    article_tokens = [(token, get_wordnet_pos(tag)) for token, tag in article_tokens] # translate to wordnet supported keys to avoid unknown key error
    lemmas += [wordnet_lemmatizer.lemmatize(token, pos=ptag) for token, ptag in article_tokens if (token not in stopwords.words('english') and len(token)>2)]
    # bag-of-lemmas from now on
    article_bow = Counter(lemmas) 
    print(article_bow)
    # 

Counter({'crypto': 7, 'fellowship': 5, 'get': 5, 'frst': 4, 'team': 4, 'next': 4, 'entrepreneur': 4, 'idea': 4, 'company': 4, 'fabric': 3, 'venture': 3, 'create': 3, 'program': 3, 'two': 3, 'firm': 3, 'want': 3, 'france': 3, 'think': 3, 'youll': 3, 'startup': 2, 'dont': 2, 'build': 2, 'look': 2, 'nearly': 2, 'already': 2, 'available': 2, 'around': 2, 'start': 2, '€100000': 2, 'one': 2, 'salary': 2, 'long': 2, 'join': 2, 'first': 2, 'isnt': 2, 'fund': 1, 'find': 1, 'might': 1, 'foster': 1, 'promising': 1, 'preconceive': 1, 'yet': 1, 'pierre': 1, 'entremont': 1, 'writes': 1, 'medium': 1, 'post': 1, 'lot': 1, 'opportunity': 1, 'success': 1, 'actively': 1, 'space': 1, 'developer': 1, 'rich': 1, 'therefore': 1, 'step': 1, 'ambition': 1, 'write': 1, 'blockchain': 1, 'development': 1, 'defi': 1, 'project': 1, 'always': 1, 'open': 1, 'source': 1, 'learning': 1, 'resource': 1, 'free': 1, 'web': 1, 'hard': 1, 'prototype': 1, 'right': 1, 'framework': 1, 'incentivize': 1, 'generation': 1, 'accept'

Counter({'musk': 10, 'say': 8, 'tweet': 7, 'doge': 6, 'send': 6, 'week': 5, 'dogecoin': 5, 'last': 4, 'tesla': 4, 'market': 4, 'bitcoin': 4, 'team': 3, 'developer': 3, 'make': 3, 'coin': 3, 'lumin': 3, 'cryptocurrency': 3, 'time': 3, 'seem': 3, 'would': 3, 'elon': 2, 'efficient': 2, 'price': 2, 'include': 2, 'joke': 2, 'plan': 2, 'month': 2, 'first': 2, 'big': 2, 'maybe': 2, 'perhaps': 2, 'worth': 2, 'create': 2, 'small': 2, 'carlson': 2, 'company': 2, 'billion': 2, 'back': 2, 'weekend': 2, 'suggest': 2, 'mention': 2, 'go': 2, 'thats': 2, 'alternative': 2, 'speak': 1, 'impact': 1, 'predictable': 1, 'moon': 1, 'late': 1, 'series': 1, 'declaration': 1, 'viral': 1, 'roller': 1, 'coaster': 1, 'past': 1, 'yet': 1, 'catch': 1, 'core': 1, 'development': 1, 'surprise': 1, 'michi': 1, 'colorado-based': 1, 'help': 1, 'maintain': 1, 'yes': 1, 'spacex': 1, 'billionaire': 1, 'spitballing': 1, 'idea': 1, 'welcome': 1, 'input': 1, 'intense': 1, 'spotlight': 1, 'recent': 1, 'string': 1, 'doge-boosting

Counter({'solana': 14, 'say': 13, 'yakovenko': 9, 'ethereum': 9, 'transaction': 9, 'network': 7, 'blockchain': 6, 'work': 6, 'big': 6, 'system': 6, 'proof': 6, 'take': 5, 'company': 5, 'new': 5, 'like': 5, 'validators': 5, 'market': 5, 'reason': 4, 'right': 4, 'miner': 4, 'cryptocurrency': 4, 'way': 4, 'developer': 4, 'stake': 4, 'people': 4, 'one': 4, 'isnt': 3, 'year': 3, 'engineer': 3, 'qualcomm': 3, 'idea': 3, 'speed': 3, 'consensus': 3, 'make': 3, 'blockchains': 3, 'thats': 3, 'call': 3, 'case': 3, 'excite': 3, 'need': 3, 'open': 3, 'faster': 3, 'certainly': 3, 'even': 3, 'good': 3, 'help': 3, 'decentralize': 3, 'go': 3, 'set': 3, 'lot': 3, 'build': 3, 'want': 3, 'global': 3, 'price': 3, 'seem': 3, 'linux': 3, 'know': 2, 'yet': 2, 'crypto': 2, 'san': 2, 'francisco': 2, 'cafe': 2, 'two': 2, 'record': 2, 'reach': 2, 'process': 2, 'bitcoins': 2, 'bitcoin': 2, 'prove': 2, 'transition': 2, 'wherein': 2, 'add': 2, 'far': 2, 'opportunity': 2, 'meanwhile': 2, 'yesterday': 2, 'mindshare': 

Counter({'project': 23, 'energy': 15, 'power': 14, 'mining': 13, 'solar': 12, 'crypto': 10, 'say': 10, 'industry': 8, 'advertisement': 8, 'montana': 7, 'state': 7, 'renewable': 7, 'would': 7, 'atlas': 7, 'build': 6, 'company': 6, 'mine': 6, 'operation': 6, 'people': 6, 'array': 5, 'standard': 5, 'million': 5, 'megawatt': 5, 'vincent': 5, 'new': 5, 'need': 5, 'hedge': 5, 'one': 4, 'country': 4, 'use': 4, 'center': 4, 'back': 4, 'sell': 4, 'utility': 4, 'seem': 4, 'local': 4, 'also': 4, 'want': 3, 'large': 3, 'instal': 3, 'cryptocurrency': 3, 'get': 3, 'really': 3, 'big': 3, 'around': 3, '2020': 3, 'call': 3, 'bitcoin': 3, 'facility': 3, 'thats': 3, 'put': 3, 'go': 3, 'forward': 3, 'renewables': 3, 'move': 3, 'owner': 3, 'charge': 3, 'cryptowatt': 3, 'impact': 2, 'could': 2, 'huge': 2, 'decarbonize': 2, 'madison': 2, 'river': 2, 'equity': 2, 'butte': 2, 'report': 2, 'developer': 2, 'take': 2, 'county': 2, 'home': 2, 'entire': 2, 'end': 2, 'rank': 2, 'solution': 2, 'lot': 2, 'permit': 2, 

Counter({'trading': 7, 'crypto': 7, 'new': 7, 'exchange': 7, 'manipulation': 6, 'platform': 6, 'work': 6, 'client': 6, 'come': 5, 'former': 5, 'financial': 5, 'institution': 5, 'alert': 5, 'know': 5, 'data': 5, 'able': 5, 'account': 5, 'solidus': 4, 'goldman': 4, 'different': 4, 'also': 4, 'demand': 4, 'broker': 4, 'dealer': 4, 'asset': 4, 'right': 4, 'way': 4, 'detect': 3, 'cryptocurrency': 3, 'today': 3, 'desk': 3, 'engineer': 3, 'build': 3, 'talk': 3, 'one': 3, 'venture': 3, 'crime': 3, 'customer': 3, 'pump': 3, 'market': 3, 'go': 3, 'like': 3, 'thing': 3, 'detection': 3, 'surface': 3, 'look': 3, 'say': 2, 'million': 2, 'fund': 2, 'government': 2, 'last': 2, 'week': 2, 'call': 2, 'compliance': 2, 'equity': 2, 'use': 2, 'case': 2, 'tool': 2, 'would': 2, 'leave': 2, 'employ': 2, 'address': 2, 'grow': 2, 'ceo': 2, 'asaf': 2, 'meir': 2, 'include': 2, 'liquidity': 2, 'provider': 2, 'regulator': 2, 'risk': 2, 'want': 2, 'largely': 2, 'wash': 2, 'dump': 2, 'increase': 2, 'think': 2, 'ask':

...

Counter({'otter': 7, 'zoom': 5, 'call': 5, 'meeting': 5, 'assistant': 5, 'say': 4, 'point': 3, 'company': 3, 'google': 3, 'work': 3, 'transcription': 3, 'feature': 3, 'also': 3, 'include': 3, 'colleague': 3, 'attend': 2, 'another': 2, 'ongoing': 2, 'audio': 2, 'software': 2, 'interface': 2, 'allow': 2, 'worry': 2, 'miss': 2, 'per': 2, 'pandemic': 1, 'safe': 1, 'would': 1, 'happy': 1, 'didnt': 1, 'ever': 1, 'covid-19': 1, 'still': 1, 'concern': 1, 'like': 1, 'move': 1, 'hybrid': 1, 'week': 1, 'fair': 1, 'video': 1, 'likely': 1, 'consistent': 1, 'fixture': 1, 'job': 1, 'foreseeable': 1, 'future': 1, 'think': 1, 'potential': 1, 'solution': 1, 'fatigue': 1, 'introduce': 1, 'new': 1, 'behalf': 1, 'take': 1, 'note': 1, 'tool': 1, 'even': 1, 'one': 1, 'youre': 1, 'host': 1, 'connect': 1, 'otterai': 1, 'microsoft': 1, 'calendar': 1, 'toggle': 1, 'want': 1, 'join': 1, 'apps': 1, 'agenda': 1, 'possible': 1, 'inputting': 1, 'url': 1, 'see': 1, 'participant': 1, 'proceeding': 1, 'cant': 1, 'preten

Counter({'game': 9, 'support': 8, 'surface': 6, 'duo': 6, 'xbox': 4, 'pas': 4, 'touch': 4, 'app': 3, 'control': 3, 'microsoft': 2, 'android': 2, 'unique': 2, 'company': 2, 'feature': 2, 'add': 2, 'announce': 1, 'officially': 1, 'dual-screen': 1, 'setup': 1, 'test': 1, 'beta': 1, 'handful': 1, 'week': 1, 'ready': 1, 'prime': 1, 'report': 1, 'windows': 1, 'central': 1, 'allow': 1, 'gamers': 1, 'play': 1, 'dedicated': 1, 'however': 1, 'unlike': 1, 'normal': 1, 'smartphone': 1, 'appear': 1, 'bottom': 1, 'display': 1, 'accord': 1, 'pcmag': 1, 'certainly': 1, 'much-needed': 1, 'appeal': 1, 'arrive': 1, 'last': 1, 'year': 1, 'underwhelming': 1, 'review': 1, 'second-screen': 1, 'essentially': 1, 'turn': 1, 'product': 1, 'handheld': 1, 'device': 1, 'similar': 1, 'nintendos': 1, '3d': 1, 'also': 1, 'say': 1, 'cloud': 1, 'xcloud': 1, 'developer': 1, 'adapt': 1, 'even': 1, 'take': 1, 'advantage': 1, 'form': 1, 'factor': 1, 'choice': 1, 'include': 1, 'wider': 1, 'aspect': 1, 'ratio': 1, 'columned':

Counter({'gate': 20, 'microsoft': 13, 'article': 12, 'advertisement': 12, 'continue': 12, 'employee': 10, 'year': 10, 'board': 10, 'say': 10, 'story': 8, 'bill': 8, 'investigation': 8, 'relationship': 7, 'load': 7, 'company': 7, 'start': 6, '2000': 6, 'yet': 6, 'time': 6, 'epstein': 6, 'report': 5, 'content': 5, 'attempt': 4, 'intimate': 4, 'involvement': 4, 'last': 4, 'step': 4, '2019': 4, 'matter': 4, 'receive': 4, 'concern': 4, 'since': 4, 'melinda': 4, 'divorce': 4, 'end': 4, 'billion': 4, 'inc': 4, 'network': 4, 'sign': 4, 'email': 4, 'share': 3, 'investigate': 3, 'earlier': 3, 'conduct': 3, 'almost': 3, 'ago': 3, 'video': 3, 'provide': 3, 'foundation': 3, 'announce': 3, 'decision': 3, 'affair': 3, 'investment': 3, 'post': 3, 'postmedia': 3, 'please': 3, 'dow': 2, 'jones': 2, 'microsofts': 2, 'director': 2, 'find': 2, 'female': 2, 'inappropriate': 2, 'decide': 2, 'via': 2, 'corp': 2, 'gatess': 2, 'two': 2, 'inform': 2, 'romantic': 2, 'person': 2, 'review': 2, 'thorough': 2, 'exter

Counter({'microsoft': 7, 'work': 5, 'employee': 5, 'remote': 4, 'hybrid': 3, 'nadella': 3, 'company': 3, 'use': 3, 'also': 3, 'network': 3, 'future': 2, 'in-person': 2, 'device': 2, 'smart': 2, 'camera': 2, 'physical': 2, 'say': 2, 'meeting': 2, 'people': 2, 'pandemic': 2, 'continue': 2, 'march': 2, 'security': 2, 'home': 2, 'launch': 1, 'playbook': 1, 'outline': 1, 'believe': 1, 'office': 1, 'near': 1, 'able': 1, 'incorporate': 1, 'help': 1, 'new': 1, 'era': 1, 'longer': 1, 'rely': 1, 'solely': 1, 'share': 1, 'location': 1, 'write': 1, 'ceo': 1, 'satya': 1, 'post': 1, 'friday': 1, 'linkedin': 1, 'doesnt': 1, 'mean': 1, 'place': 1, 'space': 1, 'arent': 1, 'important': 1, 'need': 1, 're-imagined': 1, 'order': 1, 'make': 1, 'briefing': 1, 'inclusive': 1, 'participant': 1, 'eye': 1, 'level': 1, 'high-quality': 1, 'audio': 1, 'video': 1, 'ensure': 1, 'everyone': 1, 'see': 1, 'hear': 1, 'participate': 1, 'person': 1, 'software': 1, 'team': 1, 'room': 1, 'note': 1, 'meet': 1, 'recording': 1,

Counter({'company': 5, 'codelco': 4, 'say': 4, 'operation': 4, 'copper': 3, 'microsoft': 3, 'boost': 3, 'reduce': 3, 'worker': 2, 'process': 2, 'ventanas': 2, 'mining': 2, 'automation': 2, 'improve': 2, 'project': 2, 'promote': 2, 'sustainability': 2, 'within': 2, 'many': 2, 'miner': 2, 'technology': 2, 'efficiency': 2, 'operate': 2, 'risk': 2, 'monitor': 1, 'smelter': 1, 'chile': 1, 'january': 1, '2015': 1, 'reutersrodrigo': 1, 'garridofile': 1, 'photo': 1, 'chiles': 1, 'world': 1, 'large': 1, 'producer': 1, 'monday': 1, 'strike': 1, 'deal': 1, 'tech': 1, 'giant': 1, 'fast-track': 1, 'digitization': 1, 'sprawling': 1, 'help': 1, 'extend': 1, 'analytics': 1, 'joint': 1, 'beef': 1, 'capacity': 1, 'make': 1, 'production': 1, 'efficient': 1, 'statement': 1, 'like': 1, 'chilean': 1, 'fight': 1, 'recent': 1, 'year': 1, 'adopt': 1, 'new': 1, 'keep': 1, 'cost': 1, 'age': 1, 'deposit': 1, 'suffer': 1, 'decline': 1, 'ore': 1, 'grade': 1, 'profound': 1, 'change': 1, 'different': 1, 'area': 1, 'c

In [ ]:
# collect any number-tokens as a separate token - how many numbers were used in any article's maintext?
# check spaces and - vs — 